In [ ]:
import pandas as pd
import numpy as np
import folium
import requests

from bs4 import BeautifulSoup

# pip install tqdm : progressBar 구현
# from tqdm.notebook import tqdm
from tqdm import tqdm
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

# jupyter nbconvert --to script coffeeStore.ipynb

In [ ]:
# 크롬 드라이버 다운로드 :  https://googlechromelabs.github.io/chrome-for-testing/
chrome_options = webdriver.ChromeOptions() # 크롬 브라우저 옵션
drive_path = 'chromedriver.exe' # 다운로드 받은 드라이버 파일
myservice = Service(drive_path) # 드라이버 제어를 위한 서비스 객체
driver = webdriver.Chrome(service=myservice, options=chrome_options) # 드라이버 객체
print(type(driver)) # 객체가 잘 생성되었는 지 확인

wait_time = 10 # 최대 대기 시간 10초
driver.implicitly_wait(wait_time)

In [ ]:
# driver.maximize_window()

In [ ]:
# 공용 변수 선언
dataOut = './../dataOut/'

In [ ]:
# 스타벅스) 매장 찾기-지역 검색 클릭
starbucks_url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(starbucks_url) # 해당 페이지로 이동하기

In [ ]:
# 스타벅스 '서울' 링크 클릭
starbucks_seoul_selector = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'

WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_selector))).click()

In [ ]:
# 스타벅스 '서울'-'전체' 링크 클릭
starbucks_seoul_all = '#mCSB_2_container > ul > li:nth-child(1) > a'

WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_all))).click()

In [ ]:
# 스타벅스 html 코드를 파싱해서 html 파일에 기록합니다.
html = driver.page_source # 해당 페이지 소스 코드 반환
filename = 'starbucks.html'
htmlFile = open(filename, mode='wt', encoding='UTF-8')
print(html, file=htmlFile)
htmlFile.close()
print(filename, '파일 생성됨')

In [ ]:
# 파싱된 결과를 BeautifulSoup 객체로 생성
soup = BeautifulSoup(html, 'html.parser')
print(type(soup))

In [ ]:
# id 속성이 mCSB_3_container인 <div> 태그를 찾아 주세요.
container = soup.find('div', attrs={'id': 'mCSB_3_container'})

# 그 아래에 있는 모든 <li> 태그를 찾아 주세요.
storeAll = container.find_all('li')

print(f'모든 매장 갯수 : {len(storeAll)}')

In [ ]:
starbucksData = [] # 스타벅스 매장 리스트

for store in storeAll:
    # print(store)
    brand = '스타벅스'
    name = store.find('i').text.strip()
    address = store.find('p').text.replace('1522-3232', '')
    imsi = address.split(' ')
    sido = imsi[0]
    gungu = imsi[1]
    latitude = store['data-lat']
    longitude = store['data-long']

    starbucksData.append((brand, name, address, sido, gungu, latitude, longitude))
    # break
# end for

print(f'스타벅스 매장 갯수 : {len(starbucksData)}')
# print(starbucksData)

In [ ]:
'''
<li class="quickResultLstCon"
    data-code="3762"
    data-hlytag="null"
    data-index="0"
    data-lat="37.501087"
    data-long="127.043069"
    data-name="역삼아레나빌딩"
    data-storecd="1509"
    style="background:#fff">

    <strong data-my_siren_order_store_yn="N"
            data-name="역삼아레나빌딩"
            data-store="1509"
            data-yn="N">
        역삼아레나빌딩
    </strong>

    <p class="result_details">
        서울특별시 강남구 언주로 425 (역삼동)<br/>
        1522-3232
    </p>

    <i class="pin_general">리저브 매장 2번</i>

</li>

'''

In [ ]:
sbColumns = ['브랜드', '상호', '주소', '시도', '군구', '위도', '경도']
sbDataFrame = pd.DataFrame(starbucksData)
sbDataFrame.columns = sbColumns
sbDataFrame.head()

In [ ]:
sbFilename = dataOut + 'starbucks_file.csv'
sbDataFrame.to_csv(sbFilename, index=False, encoding='UTF-8')
print(f'{sbFilename} 파일이 저장되었습니다.')

In [ ]:
# 위도, 경도 누락 데이터 갯수 확인
print(f'위도 누락 데이터 갯수 : {sbDataFrame["위도"].isnull().sum()}')
print(f'경도 누락 데이터 갯수 : {sbDataFrame["경도"].isnull().sum()}')

In [ ]:
print('# 서울시 구 갯수 파악')
seoul_gu_list = list(sbDataFrame['군구'].unique())
print(f'서울시 구 갯수: {len(seoul_gu_list)}')
print(f'\n서울시 구 목록 : {seoul_gu_list}')

In [ ]:
# 누락된 위도와 경도 정보는 kakao api를 이용하여 채워 넣도록 합니다.
# https://developers.kakao.com/
# kakao에 로그인하여 API 키 발급 받기
# 주의) 카카오 개발자 사이트에서 카카오맵-사용설정-On 상태로 변경해주세요.

In [13]:
url_header = 'https://dapi.kakao.com/v2/local/search/address.json?query='
api_key = 'a27595520f0c90739026b9f6ace4279c'
header = {'Authorization': 'KakaoAK ' + api_key}

In [8]:
# 주소를 입력 받아서 위도와 경도를 반환해주는 함수 구현하기
def getGeoCoder(address):
    result = ''
    url = url_header + address
    response = requests.get(url, headers=header)

    if response.status_code == 200:
        try:
            # print(response.json())
            jsondata = response.json()["documents"][0]["address"]
            return jsondata['y'], jsondata['x']
        except Exception as err:
            print(err)
            return None
        # end try
    else:
        result = f'Error[{response.status_code}]'
    # end if

    return result
# end def getGeoCoder

In [ ]:
# 다음 주소에 대한 테스트를 진행합니다.
address01 = '서울 중랑구 망우로 460 (망우동)'
geoInfo01 = getGeoCoder(address01) # 잘 동작하는 주소
print(geoInfo01)

address02 = '서울 노원구 한글비석로 409 (상계동) 1~2층'
geoInfo02 = getGeoCoder(address02) # NoneType이 리턴되는 주소
geoInfo02

In [ ]:
'''
케이스 01
panLatTo('127.079680860838','37.5879908042663','6');fnMove();

케이스 02
panLatTo('0','0','11');fnMove();

케이스 03
panAddTo('서울 중랑구 신내로 211 (신내동)','8');fnMove();
'''


In [ ]:
# 이디야 매장 정보
ediya_url = 'https://www.ediya.com/contents/find_store.html'
driver.get(ediya_url)

In [ ]:
# 이디야 '매장명'과 '주소' 검색에서 '주소' 버튼 클릭
ediya_address_selector = '#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a'

WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_address_selector))).click()

In [ ]:
# 서울시 구 목록을 순환하면서 크롤링
ediyaData = [] # 이디야 매장 정보 리스트
incorrect_address = [] # 위도/경도 추출에 문제가 있는 주소 리스트

ediya_search_inputbox = '#keyword' # 이디야 주소 검색 입력란
ediya_search_button = '#keyword_div > form > button' # 이디야 주소 검색 (돋보기) 버튼

for gu in tqdm(seoul_gu_list):
    # 주소 검색 입력란 초기화
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_inputbox))).clear()

    # 주소 검색어 입력 : 예시) 서울 강남구
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_inputbox))).send_keys(f'서울 {gu}')

    # 찾기(돋보기) 버튼 클릭
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ediya_search_button))).click()

    # 이디야 매장 정보 수집
    html = driver.page_source

    if gu == '마포구': # 하나의 구 정보만 저장해보기
        gufile = open(f'서울 {gu}.html', 'wt', encoding='UTF-8')
        print(html, file=gufile)
        gufile.close()
        print(f'서울 {gu}.html 파일 생성 완료')
    # end if

    soup = BeautifulSoup(html, 'html.parser')

    # 매장 정보는 id 속성이 placesList인 <ul> 태그 아래에 있습니다.
    ul_tag = soup.find('ul', id='placesList')

    oneGuStoreList = ul_tag.find_all('li') # 구별 매장 리스트

    for store in oneGuStoreList:
        # print(store)
        brand = '이디야'
        name = store.find('dt').text.strip()
        address = store.find('dd').text
        imsi = address.split(' ')
        sido = imsi[0]
        gungu = imsi[1]

        # 위도와 경도는 잠시
        geoInfoString = store.find('a')['onclick']
        # print(geoInfoString)

        latitude = np.nan # 결측치
        longitude = np.nan

        # 위도와 경도 보정
        if geoInfoString.startswith('panLatTo') :
            if geoInfoString.startswith("panLatTo('0','0'") : # 잘못된 위도/경도
                # address를 사용하여 Kakao API에서 물어 보기
                geo_info = getGeoCoder(address)
                if geo_info != None:
                    latitude, longitude = geo_info
                else:
                    incorrect_address.append(address)
                    latitude = None
                    longitude = None
                # end if
            else: # 올바른 위도/경도
                latLong = geoInfoString.split("'")
                latitude = latLong[3]
                longitude = latLong[1]
                # print(f'위도 : {latitude}, 경도 : {longitude}')
            # end if

        else: # 'panAddTo'으로 시작하는 경우
            # address를 사용하여 Kakao API에서 물어 보기
            geo_info = getGeoCoder(address)
            if geo_info != None:
                latitude, longitude = geo_info
            else:
                incorrect_address.append(address)
                latitude = None
                longitude = None
            # end if
        # end if

        ediyaData.append((brand, name, address, sido, gungu, latitude, longitude))

        # break # 주의) 차후 삭제 예정
    # end inner for

    # break # 주의) 차후 삭제 예정
# end outer for

print(f'이디야 매장 갯수 : {len(ediyaData)}')

In [ ]:
'''
<li class="item">
    <a href="#c" onclick="panLatTo('0','0','0'); fnMove();">
        <div class="store_thum">
            <img src="../images/customer/store_thum.gif" />
        </div>
        <dl>
            <dt>강남YMCA점</dt>
            <dd>서울 강남구 논현동</dd>
        </dl>
    </a>
</li>
'''

In [ ]:
edColumns = ['브랜드', '상호', '주소', '시도', '군구', '위도', '경도']
edDataFrame = pd.DataFrame(ediyaData)
edDataFrame.columns = edColumns
edDataFrame.head()

In [ ]:
print('주소 변환에 문제가 있던 항목들')
incorrect_address

In [ ]:
# 콤마를 공백으로, 2칸 공백은 1칸으로 치환합니다.
# 이후 공백으로 분리후, 슬라이싱을 사용하여 0번째 부터 3번째 글자까지만 추출합니다.
correct_address = [addr.replace(',', ' ').replace('  ', ' ').split(' ')[0:4] for addr in incorrect_address]
correct_address

In [ ]:
# split() 함수로 나누어진 주소를 join() 함수로 다시 문자열을 합칩니다.
correct_address = [' '.join(addr) for addr in correct_address]
correct_address

In [ ]:
# 보정이 된 주소 이름으로 Kakao API에게 위도/경도 추출을 요청합니다.
correction_geoinfo = [getGeoCoder(addr) for addr in correct_address]
correction_geoinfo

In [ ]:
edFilename = dataOut + 'ediya_file.csv'
edDataFrame.to_csv(edFilename, index=False, encoding='UTF-8')
print(f'{edFilename} 파일이 저장되었습니다.')

In [ ]:
# 위도, 경도 누락 데이터 갯수 확인
print(f'위도 누락 데이터 갯수 : {edDataFrame["위도"].isnull().sum()}')
print(f'경도 누락 데이터 갯수 : {edDataFrame["경도"].isnull().sum()}')

In [ ]:
# 할리스 매장

In [ ]:
hollys_base_url = 'https://www.hollys.co.kr/store/korea/korStore2.do'


In [ ]:
# 위도와 경도 추출에 문제가 있는 주소 정보들
incorrect_address = []

# 서비스 목록 이름을 저장해 놓은 전역 변수 : 타입은 list
global_service_list = []

In [27]:
# 페이지 번호를 사용하여 매장들의 정보를 10개씩 읽어 들이는 함수
def get_store_data(page_no):
    # parameters는 웹 페이지 요청시 전송할 파라미터 사전(우리는 '페이지 번호'와 '서울' 지역만 조회 예정)
    parameters = {
    'pageNo': page_no,
    'sido': '서울',
    'gugun': '',
    'store': ''
    }

    # rul에 데이터 요청 보내기
    response = requests.get(hollys_base_url, params=parameters)

    if response.status_code == 200:
        print(response.text)
        pass
    else:
        print(f'페이지 {page_no} 로딩에 실패하였습니다.')
        return []
    # end if

    soup = BeautifulSoup(response.text, 'html.parser')

    # <table> 태그의 'class' 속성이 'tb_store'인 요소를 찾습니다.
    store_table = soup.find('table', {'class': 'tb_store'})

    all_store = store_table.find_all('tr')[1:] # 상단의 table header는 제외하기

    if len(all_store) == 0: # 더이상 매장 정보가 없으면.
        return None
    # end if

    stores = [] # 현재 페이지에서 반환될 모든 매장 정보

    for one_store in all_store: # 현재 페이지의 매장 정보들을 반복
        td_tags = one_store.find_all('td')

        if len(td_tags) == 0: # 더이상 매장 정보가 없으면 무시
            continue
        # end if

        brand = '할리스'
        name = td_tags[1].text.strip()
        address = td_tags[3].text.replace(' .', '')
        imsi = address.split(' ')
        sido = imsi[0]
        gungu = imsi[1]

        # 매장 서비스 목록 : 서비스 가능한 목록은 모두 <img> 태그로 만들어져 있습니다.
        # 기본 값으로 모든 매장에서 서비스 되는 항목이 없다고 가정합니다.
        service_dict = {'테라스': 'no', '흡연시설': 'no', '주차': 'no', '24시간': 'no', 'DT 매장': 'no'}

        store_service = td_tags[4]
        # print('-'*30)
        # print(store_service)

        service_img = store_service.find_all('img')
        # print('-'*30)
        # print(f'img 태그 갯수 : {len(service_img)}')

        # 매장에서 제공되는 서비스의 key 값을 'yes'로 변경합니다.
        for image in service_img:
            alt_tag = image.get('alt', '')
            # print(alt_tag)
            service_dict[alt_tag] = 'yes'
        # end for

        # print(f'{name} 매장의 서비스 정보 확인')
        # print(service_dict)

        # 매장 서비스에 대한 사전을 정렬시켜, 각 key들의 값 정보를 추출합니다.
        service_list = sorted(service_dict.keys())
        service_data = [service_dict[key] for key in service_list]
        print(service_data) # 이 항목은 stores 리스트에 추가하도록 합니다.

        global global_service_list # global 을 적어주므로서 나는 전역 변수임을 알림
        global_service_list = service_data

        phone_number = td_tags[5].text.strip()
        if phone_number == '.':
            phone_number = None
        #end if

        # 위도와 경도에 대한 정보는 없습니다.
        # 그래서, 주소정보와 kakao Api 를 사용하여 구해야 합니다.
        geo_info = getGeoCoder(address)

        if geo_info != None:
            latitude, longitude = geo_info

        else: # 위도와 경도 변환이 안되는 주소들
            incorrect_address.append(address)
            latitude = None
            longitude = None
        # end if

        # list에 + 기호를 사용하면, 요소들을 합쳐줍니다.
        stores.append([brand, name, address, sido, gungu, latitude, longitude, phone_number] + service_data)
    # end for

    return stores
# end def get_store_data


In [28]:
hollysData = [] # 매장 정보를 저장할 리스트
page_no = 1 # 페이지 번호를 +1씩 증가하면서 크롤링

while True:
    print(f'현재 {page_no} 페이지를 크롤링 중입니다.')
    stores = get_store_data(page_no) # 페이지당 10개

    # if not stores:
    if page_no ==2:
        print('# 더이상 매장 정보가 없으므로 빠져 나가기')
        break
    # end if

    hollysData.append(stores)
    page_no += 1
# end while True


현재 1 페이지를 크롤링 중입니다.
<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="ko" lang="ko">
<head>
	<title>할리스</title>
	<meta charset="UTF-8">

	<meta name="Subject" content="HOLLYS" />
	<meta name="Title" content="HOLLYS" />
	<meta name="Keywords" content="HOLLYS,HOLLYS,할리스,할리스,커피" />
	<meta name="Description" content="할리스는 1998년 국내 첫 에스프레소 커피전문점을 개점한 순수 국내브랜드로서 당당하게 시장 선점이라는 확고한 위치를 확보하고 국내 에스프레소 커피시장을 주도해 나가고 있습니다." />
	<meta name="Author" content="HOLLYS F&B" />
	<meta name="Publisher" content="HOLLYS F&B" />
	<meta name="Classification" content="COFFEE,커피,음료,푸드" />
	<meta name="Location" content="Korea" />
	<meta name="Author-Date" content="2015.04.01" />
	<meta name="Date" content="2025.12.08" />
	<meta name="Distribution" content="HOLLYS, HOLLYS F&B" />
	<meta name="Copyright" content="HOLLYS" />
	
	<meta property="og:type" content="website">
	<meta property="og:title" content="할리스">
	<meta property="og:description" content="HOLLYS">
	<meta property="og:url" content=

In [ ]:
print(f'총 매장 갯수 : {len(hollysData)}')
hollysFrame = pd.DataFrame(hollysData)

In [ ]:
hollysFrame